In [91]:
import pandas as pd
import psycopg2 

In [92]:
conn = psycopg2.connect(host="35.200.234.61",database="sales", user="postgres", password="cx6ac54nmgGtLD1y")

In [93]:
sql="""select
	snippet.task_id,
	signal_caught.snippet_id,
	signal_caught.signal_generated_id,
	snippet.text_ as snippet_text,
	signal_generated.text_ as signal_text,
	product_signal."name",
	signal_caught.score,
	signal_caught.match_method
from
	signal_caught
left join signal_generated on
	signal_generated.id = signal_caught.signal_generated_id
left join product_signal on
	product_signal.id = signal_generated.signal_id
left join snippet on
	snippet.id = signal_caught.snippet_id
where
	signal_caught.snippet_id in (
	select
		snippet.id
	from
		snippet
	where
		snippet.task_id in(18345945,
	18345927,
	18345926))
and signal_caught.match_method = 'USE'
order by snippet.id asc"""

In [94]:
caught_singal_df=pd.read_sql_query(sql,conn)

In [95]:
caught_singal_df.rename(columns={"snippet_id":'id'},inplace=True)

In [96]:
caught_singal_df.columns.to_list()

['task_id',
 'id',
 'signal_generated_id',
 'snippet_text',
 'signal_text',
 'name',
 'score',
 'match_method']

In [97]:
caught_singal_df.drop(columns=["task_id","snippet_text"],axis=1,inplace=True)

In [98]:
sql="select * from snippet where snippet.task_id in (18345945,18345927,18345926) order by snippet.id asc"

In [99]:
snippet_df=pd.read_sql_query(sql,conn)

In [100]:
snippet_df.shape

(103, 10)

In [101]:
snippet_df.columns.to_list()

['id',
 'from_time',
 'to_time',
 'confidence',
 'text_',
 'speaker',
 'task_id',
 'language_code',
 'english_text',
 'transliterated_text']

In [102]:
snippet_df.drop(["confidence","language_code","english_text","transliterated_text"],axis=1,inplace=True)

In [103]:
snippet_df.head()

,id,from_time,to_time,text_,speaker,task_id
0,4369284,3.21,8.01,Hello.,Customer,18345926
1,4369285,5.40,6.87,Hi.,Agent,18345926
2,4369286,6.99,9.24,Good afternoon. Am I speaking with?,Agent,18345926
3,4369287,10.47,12.24,pk,Customer,18345926
4,4369288,12.99,14.40,Hi Chirag,Agent,18345926


In [104]:
out=snippet_df.merge(caught_singal_df,on="id",how="left")


In [106]:
out.to_excel("../src/resources/present_system.xlsx",index=None)

In [107]:
new_snippet=snippet_df[["speaker","text_"]].copy()

In [108]:
new_snippet.rename(columns={'text_':"text"},inplace=True).to_excel("/home/andy/Desktop/")

,speaker,text_
0,Customer,Hello.
1,Agent,Hi.
2,Agent,Good afternoon. Am I speaking with?
3,Customer,pk
4,Agent,Hi Chirag
5,Agent,I am Sushmita calling you from byju's and you ...
6,Customer,selfie
7,Agent,"so, you know"
8,Agent,Ok bye you the world's largest age.
9,Agent,Check company and byju's learning app.
